In [1]:
import pandas as pd
import os
import numpy as np

### Modfication: Look for the  ###

`PEAKMJD` is the Modified Julian Date (MJD) at which the supernova reaches peak brightness.
It is listed in the header of each `.DAT` file.

MJD is a  version of the Julian Date (JD) designed to use smaller numbers and to start the day at midnight instead of noon.

The relationship is:

$
\text{MJD} = \text{JD} - 2400000.5
$

Julian Date (JD): Counts days continuously from January 1, 4713 BCE at 12:00 TT (Julian calendar).

TT stands for **Terrestrial Time**, a modern astronomical time standard used for calcualtions.



<h3><u>Lets look at a single file and this time:</u></h3>

In [2]:
file_path = "/Users/pittsburghgraduatestudent/repos/pippin_learn/MC_BAYSN_NOSCATTER/MC_BAYSN_NOSCATTER_SN000001.DAT"

peak_mjd = None

with open(file_path, "r") as f:
    for line in f:
        if "PEAKMJD" in line:
            parts = line.strip().replace(":", "").split()
            if "PEAKMJD" in parts:
                peak_mjd = float(parts[parts.index("PEAKMJD") + 1])
                break

print(f"PEAKMJD: {peak_mjd}")

PEAKMJD: 61800.129


### Now lets look at that same single file and see if we can just filter out all the band measurements taht fall outside the +/- 7 days around the PEAKMJD ###

In [3]:
# New data rows that will be stored once filtering is done.
data_rows = []

# Opens file and reads each line.
# note: parts[1] is MJD, parts[2] is BAND, parts[4] is FLUXCAL
# data_rows is a list of tuples (MJD, BAND, FLUXCAL)
with open(file_path, "r") as f:
    for line in f:
        if line.startswith("OBS:"):
            parts = line.strip().split() #strips spaces, splits into python list.
            mjd = float(parts[1])
            band = parts[2]
            fluxcal = float(parts[4])
            data_rows.append((mjd, band, fluxcal))

# --- Filter rows within ±7 days of PEAKMJD ---
# note: row[0] is the first entry in tuple
filtered_rows = []
for row in data_rows:
    mjd_value = row[0]
    time_diff = abs(mjd_value - peak_mjd) 
    if time_diff <= 7:
        filtered_rows.append(row)

# --- Convert to a DataFrame ---
df_filtered = pd.DataFrame(filtered_rows, columns=["MJD", "BAND", "FLUXCAL"])

print(f"PEAKMJD: {peak_mjd}")
print(f"Number of near-peak rows: {len(df_filtered)}")
print(df_filtered)

PEAKMJD: 61800.129
Number of near-peak rows: 9
          MJD    BAND   FLUXCAL
0  61794.0825  LSST-r  132721.0
1  61795.0967  LSST-g  168515.0
2  61795.1033  LSST-i  109874.0
3  61795.1130  LSST-r  150376.0
4  61800.1048  LSST-r  199163.0
5  61800.1287  LSST-g  219159.0
6  61802.0967  LSST-g  218760.0
7  61802.1033  LSST-i  111339.0
8  61802.1130  LSST-r  199301.0


### Now lets see if we can filter out anything that is within +/-7 days of the peak MJD ###

In [4]:
# New data rows that will be stored once filtering is done.
# Each entry will be a tuple in the same order as VARLIST:
# (MJD, BAND, FIELD, FLUXCAL, FLUXCALERR, PHOTFLAG, GAIN, ZPT, PSF, SKY_SIG, SIM_MAGOBS)
data_rows = []

# Opens file and reads each line.
# note: parts[1] is MJD, parts[2] is BAND, parts[3] is FIELD, parts[4] is FLUXCAL, etc.
# note: MJD is measured in days. 
with open(file_path, "r") as f:
    for line in f:
        if line.startswith("OBS:"):
            parts = line.strip().split()  # strips spaces, splits into python list.

            # Extract variables in the same order as VARLIST
            mjd = float(parts[1])
            band = parts[2]
            field = parts[3]
            fluxcal = float(parts[4])
            fluxcalerr = float(parts[5])
            photflag = int(parts[6])
            gain = float(parts[7])
            zpt = float(parts[8])
            psf = float(parts[9])
            sky_sig = float(parts[10])
            sim_magobs = float(parts[11])

            # Store everything as a list of tuples.
            # Each tuple same as VARLIST order in og doc.
            data_rows.append((
                mjd, band, field, fluxcal, fluxcalerr,
                photflag, gain, zpt, psf, sky_sig, sim_magobs
            ))

# --- Filter rows within ±7 days of PEAKMJD ---
# note: row[0] is the first entry in tuple (MJD)
filtered_rows = []
for row in data_rows:
    mjd_value = row[0]
    time_diff = abs(mjd_value - peak_mjd)
    if time_diff <= 7:
        filtered_rows.append(row)

# --- Convert to a DataFrame ---
# Columns are labeled according to the VARLIST from the .DAT file
df_filtered = pd.DataFrame(
    filtered_rows,
    columns=[
        "MJD", "BAND", "FIELD", "FLUXCAL", "FLUXCALERR",
        "PHOTFLAG", "GAIN", "ZPT", "PSF", "SKY_SIG", "SIM_MAGOBS"
    ]
)

print(f"PEAKMJD: {peak_mjd}")
print(f"Number of near-peak rows: {len(df_filtered)}")
print(df_filtered)

PEAKMJD: 61800.129
Number of near-peak rows: 9
          MJD    BAND FIELD   FLUXCAL  FLUXCALERR  PHOTFLAG  GAIN    ZPT  \
0  61794.0825  LSST-r   DDF  132721.0      5.2012      4096   1.0  36.73   
1  61795.0967  LSST-g   DDF  168515.0      2.7386      6144   1.0  38.38   
2  61795.1033  LSST-i   DDF  109874.0      1.5482      4096   1.0  39.16   
3  61795.1130  LSST-r   DDF  150376.0      1.7502      4096   1.0  39.23   
4  61800.1048  LSST-r   DDF  199163.0      6.4314      4096   1.0  36.71   
5  61800.1287  LSST-g   DDF  219159.0      7.1556      4096   1.0  36.58   
6  61802.0967  LSST-g   DDF  218760.0      3.1493      4096   1.0  38.36   
7  61802.1033  LSST-i   DDF  111339.0      1.5759      4096   1.0  39.14   
8  61802.1130  LSST-r   DDF  199301.0      2.0444      4096   1.0  39.20   

    PSF  SKY_SIG  SIM_MAGOBS  
0  1.60    243.8     14.6835  
1  1.59    332.6     14.4213  
2  1.38   1156.0     14.8982  
3  1.49    786.2     14.5610  
4  2.02    246.5     14.2556  
5  2.0

### Now lets automate this to all the .dat files out there ###

Step 1 would be to define a method that collects all the near peak rows from a given folder (i.e. MC_BAYSN_NOSCATTER or MC_BAYSN_PLUSSCATTER. 

In [5]:
def collect_near_peak_rows(folder_path):
    """
    Collects all near-peak (±7 days) rows from all .DAT files in a folder.
    Returns a DataFrame with all VARLIST columns + FILENAME.
    """

    # Empty list to store all rows from all files
    all_rows = []

    # Loops through each file in the specified folder
    # and builds path to each .DAT file
    for filename in os.listdir(folder_path):
        if filename.endswith(".DAT"):
            file_path = os.path.join(folder_path, filename)


            # --- Extract PEAKMJD ---
            peak_mjd = None # Emtpy variable to store PEAKMJD
            with open(file_path, "r") as f:
                for line in f:
                    if "PEAKMJD" in line:
                        parts = line.strip().replace(":", "").split()
                        peak_mjd = float(parts[parts.index("PEAKMJD") + 1])
                        break

            # --- Extract OBS rows and filter by ±7 days ---
            with open(file_path, "r") as f:
                for line in f:
                    if line.startswith("OBS:"):
                        parts = line.strip().split()
                        mjd = float(parts[1])
                        band = parts[2]
                        field = parts[3]
                        fluxcal = float(parts[4])
                        fluxcalerr = float(parts[5])
                        photflag = int(parts[6])
                        gain = float(parts[7])
                        zpt = float(parts[8])
                        psf = float(parts[9])
                        sky_sig = float(parts[10])
                        sim_magobs = float(parts[11])

                        ## all_rows built up sequentially as we read each line.
                        if abs(mjd - peak_mjd) <= 7:
                            all_rows.append((
                                filename, mjd, band, field, fluxcal, fluxcalerr,
                                photflag, gain, zpt, psf, sky_sig, sim_magobs
                            ))

    return pd.DataFrame(
        all_rows,
        columns=[
            "FILENAME", "MJD", "BAND", "FIELD", "FLUXCAL", "FLUXCALERR",
            "PHOTFLAG", "GAIN", "ZPT", "PSF", "SKY_SIG", "SIM_MAGOBS"
        ]
    )



In [6]:
# --- STEP 1: Collect rows from both folders ---
nos_folder = "/Users/pittsburghgraduatestudent/repos/pippin_learn/MC_BAYSN_NOSCATTER"
plus_folder = "/Users/pittsburghgraduatestudent/repos/pippin_learn/MC_BAYSN_PLUSSCATTER"

nos_df = collect_near_peak_rows(nos_folder)
plus_df = collect_near_peak_rows(plus_folder)

print(f"NOSCATTER near-peak rows: {len(nos_df)}")
print(f"PLUSSCATTER near-peak rows: {len(plus_df)}")

nos_df.to_csv("nos_near_peak_rows.csv", index=False)
plus_df.to_csv("plus_near_peak_rows.csv", index=False)

print("NOSCATTER data saved to nos_near_peak_rows.csv")
print("PLUSSCATTER data saved to plus_near_peak_rows.csv")

NOSCATTER near-peak rows: 359
PLUSSCATTER near-peak rows: 363
NOSCATTER data saved to nos_near_peak_rows.csv
PLUSSCATTER data saved to plus_near_peak_rows.csv


### Sorted by FILENAME and MJD to Better Explore the Code Using Human Eyes ###

In [7]:
# Keep nos_df unchanged, create a sorted copy
sorted_nos_df = nos_df.sort_values(by=["FILENAME", "MJD"]).reset_index(drop=True)

# Save sorted DataFrame to CSV
sorted_nos_df.to_csv("sorted_nos_near_peak_rows.csv", index=False)
print("Sorted NOSCATTER data saved to sorted_nos_near_peak_rows.csv")

# Keep plus_df unchanged, create a sorted copy
sorted_plus_df = plus_df.sort_values(by=["FILENAME", "MJD"]).reset_index(drop=True)

# Save sorted DataFrame to CSV
sorted_plus_df.to_csv("sorted_plus_near_peak_rows.csv", index=False)
print("Sorted PLUSSCATTER data saved to sorted_plus_near_peak_rows.csv")

Sorted NOSCATTER data saved to sorted_nos_near_peak_rows.csv
Sorted PLUSSCATTER data saved to sorted_plus_near_peak_rows.csv


### Found Out the cause of the Mismatch ###

The individual FILENAME of each row, as part of the following .csv files contain `NOSCATTER` and `PLUSSCATTER` in them. Making it impossible to match any rows together even if matches exist. 

To solve this problem, I simply renamed the `FILENAME` of each .DAT file to contain only the SN identifier. 

For example: `MC_BAYSN_PLUSSCATTER_SN000011.DAT` -> `SN000011`

This way when I try to Merge data files, I can use `FILENAME`, `MJD`, and `BAND` to precisely allign data together and ultimately: do computation on it. 

NOTE: this means that I have to keep careful track weather my .csv files are from the dataset that contained scatter vs. a data set that did not contain scatter. This is solved by the _NO_S and _PLUS_S header addendums in the output file: `merged_near_peak_rows.csv`. 


In [8]:
# extracts the SN identifier from FILENAME
# SN - literal text match
# \d+ - matches one or more digits after the SN
# () - captures only this group and no other file name. 

nos_df["FILENAME"] = nos_df["FILENAME"].str.extract(r"(SN\d+)")
plus_df["FILENAME"] = plus_df["FILENAME"].str.extract(r"(SN\d+)")

merged_df = pd.merge(
    nos_df,
    plus_df,
    on=["FILENAME", "MJD", "BAND"],
    suffixes=("_NO_S", "_PLUS_S"),
    how="inner"
)

merged_df.to_csv("merged_near_peak_rows.csv", index=False)
print("Merged data saved to merged_near_peak_rows.csv")

Merged data saved to merged_near_peak_rows.csv


### Rows that Could not Be Matched ###

Introducing scatter  ends up changing the PEAKMJD for individual .DAT files. This causes entreis in both NOSCATTER and PLUSSCATTER data files that are within +/- 7 days for indivudal data files but do not match entries in their counterpart as PEAKMDJDS will not cover the same ranges of MJD's between dat afiles. 

For an example of this take a look at The PEAKMJD entry in the `SN000016` .DAT files.

In [9]:
compare_df = pd.merge(
    nos_df,
    plus_df,
    on=["FILENAME", "MJD", "BAND"],
    suffixes=("_NO_S", "_PLUS_S"),
    how="outer",
    indicator=True  # Adds a column showing merge source
)

# --- Extract missing rows  NOSCATTER  PLUSSCATTER ---
# note: left means the first DataFrame (nos_df)
# note: right means the second DataFrame (plus_df)
left_only = compare_df[compare_df["_merge"] == "left_only"]
right_only = compare_df[compare_df["_merge"] == "right_only"]

# --- Print results ---

print(" No Match Rows: ")

print("\nRows only in NOSCATTER (left_only):")
print(left_only[["FILENAME", "MJD", "BAND"]])

print("\nRows only in PLUSSCATTER (right_only):")
print(right_only[["FILENAME", "MJD", "BAND"]])

compare_df.to_csv("compare_near_peak_rows.csv", index=False)


 No Match Rows: 

Rows only in NOSCATTER (left_only):
     FILENAME         MJD    BAND
85   SN000016  60990.1218  LSST-r
132  SN000025  61474.0242  LSST-g
133  SN000025  61474.0308  LSST-i
155  SN000038  60876.3473  LSST-g
156  SN000038  60876.3550  LSST-i
157  SN000038  60876.3647  LSST-r

Rows only in PLUSSCATTER (right_only):
     FILENAME         MJD    BAND
124  SN000025  61456.0911  LSST-r
125  SN000025  61456.1147  LSST-i
158  SN000038  60886.2919  LSST-r
159  SN000038  60886.3158  LSST-i
160  SN000038  60890.2040  LSST-g
161  SN000038  60890.2116  LSST-i
162  SN000038  60890.2213  LSST-r
326  SN000086  61379.1589  LSST-g
327  SN000086  61379.1655  LSST-i
328  SN000086  61379.1752  LSST-r


### Separating Our Merged Data into df that have individual g, r, i bands ###



In [10]:
# Separate into three DataFrames based on BAND
merged_g_df = merged_df[merged_df["BAND"].str.contains("LSST-g")]
merged_r_df = merged_df[merged_df["BAND"].str.contains("LSST-r")]
merged_i_df = merged_df[merged_df["BAND"].str.contains("LSST-i")]

### Converting FLUXCAL to Magnitudes

We use the standard magnitude conversion:

$
m = 27.5 - 2.5 \log_{10}(\text{FLUXCAL})
$

Note: 27.5 is the zero point magnitude.

In [11]:
merged_df["MAG_NO_S"] = 27.5 - 2.5 * np.log10(merged_df["FLUXCAL_NO_S"])
merged_df["MAG_PLUS_S"] = 27.5 - 2.5 * np.log10(merged_df["FLUXCAL_PLUS_S"])

### RMS Scatter of Magnitudes

$\huge \sigma_{\text{scatter}} = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(m_{+,i} - m_{-,i})^2}$  

Where:  
- **$\sigma_{\text{scatter}}$** = RMS deviation between scatter and no-scatter magnitudes  
- **$N$** = number of observations (data points)  
- **$m_{+,i}$** = magnitude with scatter for the $i^{\text{th}}$ observation  
- **$m_{-,i}$** = magnitude without scatter for the $i^{\text{th}}$ observation  

**Conceptually:** We are measuring **how far the scatter simulation deviates from the no-scatter (baseline) magnitudes** on average.  

In [12]:
# --- Compute Δm = m_plus - m_no ---
merged_df["DELTA_M"] = merged_df["MAG_PLUS_S"] - merged_df["MAG_NO_S"]

# --- Compute RMS per band ---
rms_results = []

for band in ["LSST-g", "LSST-r", "LSST-i"]:
    band_mask = merged_df["BAND"].str.contains(band)
    band_data = merged_df.loc[band_mask, "DELTA_M"]
    N = len(band_data)
    rms = np.sqrt(np.mean(band_data**2))
    rms_results.append((band, N, rms))

# Print RMS results
print("\nRMS Results:")
for band, N, rms in rms_results:
    print(f"RMS_{band}: {rms:.6f} (N = {N})")


RMS Results:
RMS_LSST-g: 0.130852 (N = 83)
RMS_LSST-r: 0.123455 (N = 133)
RMS_LSST-i: 0.121236 (N = 137)
